In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
import os

In [2]:
df1 = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May3rd1percent_RLV_data 100 points2 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
df2 = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May3rd1percent_RLV_data 100 points2_after428 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
df3 = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May3rd1percent_RLV_data 100 points2_after450 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
df = pd.DataFrame()
df = pd.concat([df1,df2,df3],ignore_index=True)
df3=pd.DataFrame(df.iloc[:,0:3])
input = df3.values
output = df.values 
X = input[:,0:]
y = output[:,7:9]
# Individual Data
h = X[:,0]
v = X[:,1]
s = X[:,2]
# t = X[:,3]
T = y[:,0]
b = y[:,1]

In [3]:
# 10 percent of total data is takein as cross validation size 
cv_size = int(X.shape[0]*0.1)

from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.1, random_state=42)
X_val = X_train[-cv_size:]
y_val = y_train[-cv_size:]
X_train = X_train[:-cv_size]
y_train = y_train[:-cv_size]

def create_model(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(3,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'tanh')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

batchsize = 32
STEPS_PER_EPOCH = X_train.shape[0] / batchsize
save_period = 5
save_period*STEPS_PER_EPOCH

lr = 0.001

for epoch in range(0,1100,50):
    lr = lr_sch(epoch,lr)
    print(lr)

## Trying for batch size 64 with learning rate modifying in two ways for batch size 64

In [12]:
6.72055175527930e-05

6.7205517552793e-05

In [16]:
i = 8
model = create_model(2**i)
model.load_weights(f"/home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons{2**i}/cp.ckpt")
opt = tf.keras.optimizers.Adam(learning_rate = 6.72055175527930e-05)
loss = tf.keras.losses.mean_squared_error
def lr_sch(epoch,lr):
    if (epoch<300):
      if epoch%100==0: # for every 100 epochs the learning rate varies as metioned. 
        return lr*np.exp(-0.45)
      else:
        return lr
    elif(epoch>300):
        if epoch%100==0:
            return lr*np.exp(-0.225)
        else:
            return lr
    else:
        return lr
    
def lr_sch3(epoch,lr):
    if (epoch >0) &(epoch<400):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return lr*np.exp(-0.45)
      else:
        return lr
    elif(epoch>400):
        if epoch%50==0:
            return lr*0.9;
        else:
            return lr
    else:
        return lr
def lr_sch4(epoch,lr):
    epoch = epoch+339
    print("the actual epoch is", epoch)
    if (epoch >0) &(epoch<400):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return lr*np.exp(-0.45)
      else:
        return lr
    elif(epoch>400):
        if epoch%50==0:
            return lr*0.9;
        else:
            return lr
    else:
        return lr

print("The Number of Hidden units used is: ",2**i)
# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch,verbose = 0)


checkpoint_path = f"/home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons{2**i}/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# BATCH_SIZE = 128
bs = 64
STEPS_PER_EPOCH = X_train.shape[0] / bs
save_period = 5

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1,save_freq=int(save_period*STEPS_PER_EPOCH))

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch4,verbose = 1)
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
# removing the checkpoint callback
hist = model.fit(X_train,y_train,epochs = 661,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback],validation_data=(X_val, y_val))

The Number of Hidden units used is:  256
the actual epoch is 339

Epoch 1: LearningRateScheduler setting learning rate to 6.720551755279303e-05.
Epoch 1/661


2022-05-11 17:39:21.024127: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open /home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons256/cp.ckpt: FAILED_PRECONDITION: /home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons256/cp.ckpt; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


2762/2762 [==============================] - 17s 6ms/step - loss: 7.6909e-04 - accuracy: 0.9745 - val_loss: 9.0371e-04 - val_accuracy: 0.9710 - lr: 6.7206e-05
the actual epoch is 340

Epoch 2: LearningRateScheduler setting learning rate to 6.720551755279303e-05.
Epoch 2/661
2762/2762 [==============================] - 16s 6ms/step - loss: 7.7853e-04 - accuracy: 0.9737 - val_loss: 0.0010 - val_accuracy: 0.9673 - lr: 6.7206e-05
the actual epoch is 341

Epoch 3: LearningRateScheduler setting learning rate to 6.720551755279303e-05.
Epoch 3/661
2762/2762 [==============================] - 16s 6ms/step - loss: 7.8254e-04 - accuracy: 0.9741 - val_loss: 7.3322e-04 - val_accuracy: 0.9766 - lr: 6.7206e-05
the actual epoch is 342

Epoch 4: LearningRateScheduler setting learning rate to 6.720551755279303e-05.
Epoch 4/661
2762/2762 [==============================] - 16s 6ms/step - loss: 7.7605e-04 - accuracy: 0.9742 - val_loss: 8.2242e-04 - val_accuracy: 0.9750 - lr: 6.7206e-05
the actual epoch is 

Epoch 27/661
2762/2762 [==============================] - 16s 6ms/step - loss: 7.3511e-04 - accuracy: 0.9757 - val_loss: 8.4214e-04 - val_accuracy: 0.9739 - lr: 4.2852e-05
the actual epoch is 366

Epoch 28: LearningRateScheduler setting learning rate to 4.2852130718529224e-05.
Epoch 28/661
2762/2762 [==============================] - 16s 6ms/step - loss: 7.4076e-04 - accuracy: 0.9753 - val_loss: 7.5357e-04 - val_accuracy: 0.9766 - lr: 4.2852e-05
the actual epoch is 367

Epoch 29: LearningRateScheduler setting learning rate to 4.2852130718529224e-05.
Epoch 29/661
2762/2762 [==============================] - 16s 6ms/step - loss: 7.2929e-04 - accuracy: 0.9764 - val_loss: 9.1385e-04 - val_accuracy: 0.9657 - lr: 4.2852e-05
the actual epoch is 368

Epoch 30: LearningRateScheduler setting learning rate to 4.2852130718529224e-05.
Epoch 30/661
2731/2762 [============================>.] - ETA: 0s - loss: 7.3471e-04 - accuracy: 0.9756
Epoch 30: saving model to /home/saichaitanya/Chaitanya/for acc

Epoch 53/661
2762/2762 [==============================] - 16s 6ms/step - loss: 7.2983e-04 - accuracy: 0.9761 - val_loss: 8.7165e-04 - val_accuracy: 0.9712 - lr: 4.2852e-05
the actual epoch is 392

Epoch 54: LearningRateScheduler setting learning rate to 4.2852130718529224e-05.
Epoch 54/661
2762/2762 [==============================] - 16s 6ms/step - loss: 7.2444e-04 - accuracy: 0.9761 - val_loss: 7.4188e-04 - val_accuracy: 0.9758 - lr: 4.2852e-05
the actual epoch is 393

Epoch 55: LearningRateScheduler setting learning rate to 4.2852130718529224e-05.
Epoch 55/661
2705/2762 [============================>.] - ETA: 0s - loss: 7.3141e-04 - accuracy: 0.9759
Epoch 55: saving model to /home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons256/cp.ckpt
2762/2762 [==============================] - 16s 6ms/step - loss: 7.3219e-04 - accuracy: 0.9759 - val_loss: 8.0653e-04 - val_accuracy: 0.9761 - lr: 4.2852e-05
the actual epoch is 394

Epoch 56: LearningRateScheduler setting learn

...

Epoch 544/661
2762/2762 [==============================] - 19s 7ms/step - loss: 6.7433e-04 - accuracy: 0.9783 - val_loss: 6.9302e-04 - val_accuracy: 0.9785 - lr: 1.6602e-05
the actual epoch is 883

Epoch 545: LearningRateScheduler setting learning rate to 1.6601794413872994e-05.
Epoch 545/661
2210/2762 [=======================>......] - ETA: 3s - loss: 6.6906e-04 - accuracy: 0.9787
Epoch 545: saving model to /home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons256/cp.ckpt
2762/2762 [==============================] - 19s 7ms/step - loss: 6.7405e-04 - accuracy: 0.9785 - val_loss: 7.4062e-04 - val_accuracy: 0.9757 - lr: 1.6602e-05
the actual epoch is 884

Epoch 546: LearningRateScheduler setting learning rate to 1.6601794413872994e-05.
Epoch 546/661
2762/2762 [==============================] - 19s 7ms/step - loss: 6.7076e-04 - accuracy: 0.9786 - val_loss: 6.9191e-04 - val_accuracy: 0.9802 - lr: 1.6602e-05
the actual epoch is 885

Epoch 547: LearningRateScheduler settin

Epoch 570/661
2187/2762 [======================>.......] - ETA: 3s - loss: 6.5816e-04 - accuracy: 0.9794
Epoch 570: saving model to /home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons256/cp.ckpt
2762/2762 [==============================] - 20s 7ms/step - loss: 6.6074e-04 - accuracy: 0.9794 - val_loss: 6.8149e-04 - val_accuracy: 0.9807 - lr: 1.4942e-05
the actual epoch is 909

Epoch 571: LearningRateScheduler setting learning rate to 1.4941614608687814e-05.
Epoch 571/661
2762/2762 [==============================] - 19s 7ms/step - loss: 6.6885e-04 - accuracy: 0.9788 - val_loss: 6.9577e-04 - val_accuracy: 0.9808 - lr: 1.4942e-05
the actual epoch is 910

Epoch 572: LearningRateScheduler setting learning rate to 1.4941614608687814e-05.
Epoch 572/661
2762/2762 [==============================] - 19s 7ms/step - loss: 6.6420e-04 - accuracy: 0.9792 - val_loss: 9.4011e-04 - val_accuracy: 0.9675 - lr: 1.4942e-05
the actual epoch is 911

Epoch 573: LearningRateScheduler settin

2762/2762 [==============================] - 19s 7ms/step - loss: 6.6456e-04 - accuracy: 0.9790 - val_loss: 7.1416e-04 - val_accuracy: 0.9793 - lr: 1.4942e-05
the actual epoch is 934

Epoch 596: LearningRateScheduler setting learning rate to 1.4941614608687814e-05.
Epoch 596/661
2762/2762 [==============================] - 19s 7ms/step - loss: 6.6584e-04 - accuracy: 0.9788 - val_loss: 6.9647e-04 - val_accuracy: 0.9795 - lr: 1.4942e-05
the actual epoch is 935

Epoch 597: LearningRateScheduler setting learning rate to 1.4941614608687814e-05.
Epoch 597/661
2762/2762 [==============================] - 20s 7ms/step - loss: 6.6256e-04 - accuracy: 0.9791 - val_loss: 8.5143e-04 - val_accuracy: 0.9703 - lr: 1.4942e-05
the actual epoch is 936

Epoch 598: LearningRateScheduler setting learning rate to 1.4941614608687814e-05.
Epoch 598/661
2762/2762 [==============================] - 20s 7ms/step - loss: 6.6484e-04 - accuracy: 0.9792 - val_loss: 7.5240e-04 - val_accuracy: 0.9761 - lr: 1.4942e-05
t

Epoch 621/661
2762/2762 [==============================] - 20s 7ms/step - loss: 6.6103e-04 - accuracy: 0.9791 - val_loss: 7.0591e-04 - val_accuracy: 0.9787 - lr: 1.3447e-05
the actual epoch is 960

Epoch 622: LearningRateScheduler setting learning rate to 1.3447453056869563e-05.
Epoch 622/661
2762/2762 [==============================] - 20s 7ms/step - loss: 6.5977e-04 - accuracy: 0.9793 - val_loss: 6.8823e-04 - val_accuracy: 0.9800 - lr: 1.3447e-05
the actual epoch is 961

Epoch 623: LearningRateScheduler setting learning rate to 1.3447453056869563e-05.
Epoch 623/661
2762/2762 [==============================] - 20s 7ms/step - loss: 6.5905e-04 - accuracy: 0.9793 - val_loss: 6.9112e-04 - val_accuracy: 0.9809 - lr: 1.3447e-05
the actual epoch is 962

Epoch 624: LearningRateScheduler setting learning rate to 1.3447453056869563e-05.
Epoch 624/661
2762/2762 [==============================] - 19s 7ms/step - loss: 6.6397e-04 - accuracy: 0.9789 - val_loss: 6.8515e-04 - val_accuracy: 0.9786 - lr

Epoch 647/661
2762/2762 [==============================] - 20s 7ms/step - loss: 6.6106e-04 - accuracy: 0.9790 - val_loss: 7.1657e-04 - val_accuracy: 0.9781 - lr: 1.3447e-05
the actual epoch is 986

Epoch 648: LearningRateScheduler setting learning rate to 1.3447453056869563e-05.
Epoch 648/661
2762/2762 [==============================] - 19s 7ms/step - loss: 6.5845e-04 - accuracy: 0.9790 - val_loss: 6.9447e-04 - val_accuracy: 0.9785 - lr: 1.3447e-05
the actual epoch is 987

Epoch 649: LearningRateScheduler setting learning rate to 1.3447453056869563e-05.
Epoch 649/661
2762/2762 [==============================] - 19s 7ms/step - loss: 6.6347e-04 - accuracy: 0.9792 - val_loss: 6.8951e-04 - val_accuracy: 0.9784 - lr: 1.3447e-05
the actual epoch is 988

Epoch 650: LearningRateScheduler setting learning rate to 1.3447453056869563e-05.
Epoch 650/661
2107/2762 [=====================>........] - ETA: 4s - loss: 6.6301e-04 - accuracy: 0.9787
Epoch 650: saving model to /home/saichaitanya/Chaitanya

lr = 6.72055175527930e-05

In [13]:
model.save(f"/home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons{2**i}/cp.ckpt")

INFO:tensorflow:Assets written to: /home/saichaitanya/Chaitanya/for accuracy of model /batch size 64 Neurons256/cp.ckpt/assets


In [17]:
model.predict(X_test)

array([[0.19758138, 0.48758274],
       [0.64430076, 0.49762306],
       [0.10744065, 0.71280205],
       ...,
       [0.657637  , 0.49755102],
       [0.68621355, 0.49105304],
       [0.07788485, 0.65001696]], dtype=float32)

In [18]:
y_test

array([[0.199722  , 0.48816747],
       [0.64228165, 0.49793313],
       [0.10720997, 0.7140712 ],
       ...,
       [0.65561533, 0.49777708],
       [0.66030014, 0.49746499],
       [0.08146928, 0.65325258]])

def lr_sch2(epoch,lr):
    if (epoch>0) & (epoch<200):
      if epoch%5==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>20):
        if epoch%20==0:
            return round(lr*np.exp(-0.125),7)
        else:
            return round(lr,7)
    else:
        return round(lr,7)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch2,verbose = 1)
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
hist = model.fit(X_train,y_train,epochs = 10,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler],validation_data=(X_val, y_val))

## It is observed that if we get 95 % accuracy model we can directly use lower learning rates of 1 order less to reduce the time. But, maximum accuracy acheived is 97.75 @ lr = 2.5e-6 

## Testing in seperate model

In [ ]:
lr = 0.00064

In [ ]:
model2 = create_model(256)
model2.load_weights(f"/home/saichaitanya/Chaitanya/for accuracy of model /{2**i}/cp.ckpt")

In [ ]:
lr = round(lr_sch(100,lr),6)
opt = tf.keras.optimizers.Adam(learning_rate = lr)

model2.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
history = model2.fit(X_train,y_train,epochs = 80,batch_size = batchsize,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback])
model2.evaluate(X_test,y_test)

In [ ]:
df.describe()

In [ ]:
history.history.keys()

In [ ]:
hist_0001 = history.history.get('loss')


In [ ]:
lr =0.01

In [ ]:
lr = round(lr_sch(100,lr),6)
opt = tf.keras.optimizers.Adam(learning_rate = float(lr))
lr

# Cheking if round of error is the reason for fluctuation in model accuracy

In [ ]:
model3 = create_model(256)

opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
model3.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
history = model3.fit(X_train,y_train,epochs = 80,batch_size = 32,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler])
model3.save(/home/saichaitanya/Chaitanya/)

In [ ]:
model3.save('/home/saichaitanya/Chaitanya/')
model3.load_weights('/home/saichaitanya/Chaitanya/')

In [ ]:
def lr_sch2(epoch,lr):
    if (epoch>0) & (epoch<200):
      if epoch%5==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>20):
        if epoch%20==0:
            return round(lr*np.exp(-0.125),7)
        else:
            return round(lr,7)
    else:
        return round(lr,7)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch2,verbose = 1)
model3.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
history = model3.fit(X_train,y_train,epochs = 80,batch_size = 32,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler])
# model3.save(/home/saichaitanya/Chaitanya/)